# CityBikes

Send a request to CityBikes for the city of your choice. 

In [196]:
import requests 
from IPython.display import JSON
import json
import pandas as pd

In [197]:
URL = 'http://api.citybik.es/v2/networks'
response_all = requests.get(URL, headers={'accept': 'application/JSON'})
res_all = response_all.json()
JSON(res_all)

<IPython.core.display.JSON object>

In [198]:
#find out all the networks in the city of 'London' managing city bike stations.
network_id = []
for network in res_all["networks"]:
    if network["location"]["city"] == "London":
        network_id.append(network["id"])

print(network_id)

['santander-cycles']


Parse through the response to get the details you want for the bike stations in that city (latitude, longitude, number of bikes). 

In [199]:
url = 'http://api.citybik.es/v2/networks/' + network_id[0]
response_city = requests.get(url, headers={'accept': 'application/JSON'})
res_city = response_city.json()

JSON(res_city)

<IPython.core.display.JSON object>

Put your parsed results into a DataFrame.

In [200]:
#Get all the bike stations belonging to the city of London and collate them into a csv file

df_normalize = pd.json_normalize(
    res_city, 
    record_path=['network', 'stations'], 
    meta=[['network', 'id'],
          ['network', 'location', 'city'],
         ]
)

df_citybikes = df_normalize [['network.location.city', 'network.id', 'name', 'id', 
                              'latitude', 'longitude', 'empty_slots', 'free_bikes']].copy()

df_citybikes.rename(columns = {'network.location.city':'network_city', 'network.id':'network_name', 
                               'name':'station_name', 'id':'station_id', 
                               'latitude':'station_latitude', 'longitude':'station_longitude'}, 
                    inplace = True)

df_citybikes['total_bikes'] = df_citybikes['empty_slots'] + df_citybikes['free_bikes']

print(df_citybikes.shape)
print(df_citybikes.info())

#Total number of bike stations discovered and collated: 796
#CSV file name: ’london_city_bikes_stns.csv’

(796, 9)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 796 entries, 0 to 795
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   network_city       796 non-null    object 
 1   network_name       796 non-null    object 
 2   station_name       796 non-null    object 
 3   station_id         796 non-null    object 
 4   station_latitude   796 non-null    float64
 5   station_longitude  796 non-null    float64
 6   empty_slots        796 non-null    int64  
 7   free_bikes         796 non-null    int64  
 8   total_bikes        796 non-null    int64  
dtypes: float64(2), int64(3), object(4)
memory usage: 56.1+ KB
None


In [201]:
df_citybikes.to_csv("london_city_bikes_stns.csv", index=False)